How to run a keras model over multiple GPU and get speedups.

In [ ]:
!git clone https://github.com/vlimant/mpi_learn
!cp mpi_learn/mpiLAPI .

In [1]:
from mpiLAPI import mpi_learn_api

Using TensorFlow backend.


In [2]:
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Input, Reshape, Conv2D

In [3]:
activation='sigmoid'

In [4]:
i = Input((28,28,1))
l = Reshape( (28*28,))(i)
l = Dense(1000, activation=activation)(l)
o = Dense(10, init='uniform', activation='softmax')(l)
simple = Model( i ,o )
simple.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [5]:
import glob
files = list(glob.glob('/data/shared/mnist/mnist_t*.h5'))

In [6]:
mlapi = mpi_learn_api( model = simple,
                        train_files = files[:-4],
                        val_files = files[-4:],
                        check_file = True
)

In [7]:
o = mlapi.train(N=4,
                trial_name = 'test',
                features_name = 'features',
                labels_name = 'labels',
                batch = 10,
                epoch = 10,
                verbose = True,
                loss = 'categorical_crossentropy',
                easgd = False,
                early_stopping = 5,
                tf= True
)

mpirun  -n 4  mpi_learn/MPIDriver.py /tmp//8d3b33ef046728eaea8e1bc79dc60404.json /tmp//8d3b33ef046728eaea8e1bc79dc60404_train.list /tmp//8d3b33ef046728eaea8e1bc79dc60404_val.list --tf --labels-name labels --master-gpu --features-name features --max-gpus 8 --trial-name test --epoch 10 --batch 10


In [8]:
!ls

2157338924cc847a717edc818bfe2094_test_mpi_learn_result.h5
6759964687cb843761250b200032a119_test_mpi_learn_result.h5
8d3b33ef046728eaea8e1bc79dc60404_test_history.json
8d3b33ef046728eaea8e1bc79dc60404_test_mpi_learn_result.h5
algo_.pkl
d60ac6cdf044f5eb45aa72b0b707ffa7_test_mpi_learn_result.h5
deeb957daac5be1763cec3041925800a_test_mpi_learn_result.h5
mpiLAPI.py~
mpiLAPI.pyc
mpi_learn
__pycache__
README.md
run_train_ex.py
savedd.py
threaded_skopt.py
threaded_skopt.py~
threaded_skopt.pyc
tuto-1-Jupyter.ipynb
tuto-2-datasets.ipynb
tuto-3-matplotlib.ipynb
tuto-4-scikit-learn.ipynb
tuto-5-keras.ipynb
tuto-6-skopt.ipynb
tuto-7-mpiAPI.ipynb


you won't particularly get a speedup with this model because of how trivial it is, and the worker/master communication is likely overwhelming. It does provide some speedup on more complicated models/dataset https://arxiv.org/abs/1712.05878 

More information about this experimental package available at https://github.com/duanders/mpi_learn
You are better off running the training in the command line using mpirun